# AI 기반 이메일 자동 분류 및 응답 시스템

## 1. 프로젝트 개요
이 시스템은 기업의 이메일 커뮤니케이션을 효율적으로 관리하기 위한 AI 기반 자동화 솔루션입니다. GPT를 활용하여 수신된 이메일의 내용을 분석하고, 적절한 응답 템플릿을 제공함으로써 이메일 처리 시간을 단축하고 응답 품질을 일관되게 유지합니다.

## 2. 핵심 기능
1. **이메일 자동 분류**
   - 6가지 카테고리로 분류 (고객문의, 기술지원, 영업요청, 내부협업, 회의일정, 기타)
   - 4단계 우선순위 판단 (긴급, 높음, 중간, 낮음)
   - GPT 기반 컨텍스트 이해 및 분석

2. **맞춤형 응답 생성**
   - 유형별/우선순위별 차별화된 응답 템플릿
   - 상황에 맞는 커뮤니케이션 톤 조절
   - 처리 시간 명시 및 후속 절차 안내

## 3. 서비스 흐름도

```
[사용자] → [시스템] → [GPT] → [템플릿 DB]
   ↑          ↓         ↓          ↓
   └──────────┴─────────┴──────────┘
```

### 처리 단계별 상세 흐름

```
1. **이메일 수신 단계**
   사용자 → 시스템
   - 이메일 내용 전달
   - 발신자 정보 포함

2. **내용 분석 단계**
   시스템 → GPT
   - 이메일 컨텐츠 분석 요청
   - 시스템 프롬프트 전달

3. **GPT 분석 수행**
   GPT 엔진 작업:
   - 이메일 유형 분류 (6가지)
   - 우선순위 판단 (4단계)
   - 판단 근거 생성

4. **템플릿 매칭 단계**
   시스템 → 템플릿 DB
   - 분석 결과에 따른 템플릿 검색
   - 적절한 응답 양식 선택

5. **최종 응답 생성**
   시스템 → 사용자
   - 분석 결과 포함
   - 응답 템플릿 적용
   - 처리 시간 기록
```

## 4. 주요 컴포넌트 설명

1. **GPT 분석 엔진 (`analyze_email_with_gpt`)**
   ```python
   입력: 이메일 내용, 발신자 정보
   출력: {
       "type": "이메일 유형",
       "priority": "우선순위",
       "reasoning": "판단 근거"
   }
   ```

2. **응답 템플릿 관리 (`EMAIL_TEMPLATES`)**
   - 각 유형 및 우선순위별 맞춤 응답 저장
   - 총 24개의 상황별 템플릿 (6개 유형 × 4개 우선순위)

3. **메인 처리 로직 (`process_email`)**
   ```python
   결과: {
       "analysis": 이메일 분석 결과,
       "response_template": 응답 템플릿,
       "timestamp": 처리 시간
   }
   ```

## 5. 처리 예시
```python
입력 이메일:
"""
제목: 서비스 장애 발생
내용: 결제 시스템 오류 발생...
"""

분석 결과:
- 유형: 기술지원
- 우선순위: 긴급
- 근거: 시스템 장애, 고객 영향도

응답 템플릿:
"문제 상황을 확인했습니다. 기술팀이 즉시 조치를 시작했으며..."
```

## 6. 기대 효과

1. **업무 효율성 향상**
   - 이메일 분류 자동화
   - 응답 시간 단축
   - 일관된 응답 품질 유지

2. **리스크 관리**
   - 중요 메일 누락 방지
   - 우선순위에 따른 체계적 처리
   - 응답 품질 표준화

3. **확장성**
   - 새로운 이메일 유형 추가 용이
   - 템플릿 커스터마이징 가능
   - API 연동 가능

## 7. 향후 개선 방향

1. **고도화 방안**
   - 다국어 지원
   - 더 세분화된 분류 체계
   - 맞춤형 응답 자동 생성

2. **통합 확장**
   - 이메일 클라이언트 연동
   - 티켓팅 시스템 연동
   - 업무 자동화 도구 연동


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

#API-KEY 설정
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
#모델 설정
GPT_MODEL = 'gpt-3.5-turbo'

In [3]:
from openai import OpenAI
import json
from datetime import datetime
from typing import List, Dict

# OpenAI 클라이언트 설정
client = OpenAI(api_key=OPENAI_API_KEY)
GPT_MODEL = "gpt-3.5-turbo"

# 이메일 분석을 위한 시스템 프롬프트
ANALYSIS_PROMPT = """당신은 이메일 내용을 분석하여 유형과 우선순위를 판단하는 전문가입니다.
다음 기준에 따라 이메일을 분석하고 JSON 형태로 응답해주세요:

이메일 유형:
- 고객문의: 제품/서비스 관련 문의
- 기술지원: 기술적 문제나 장애 관련
- 영업요청: 구매, 견적, 제안 관련
- 내부협업: 사내 업무 협조 요청
- 회의일정: 미팅 관련 요청
- 기타: 위 카테고리에 속하지 않는 경우

우선순위:
- 긴급: 즉시 대응 필요 (시스템 장애, 긴급 고객 대응 등)
- 높음: 당일 처리 필요
- 중간: 2-3일 내 처리 가능
- 낮음: 일주일 내 처리 가능

응답 형식:
{
    "type": "이메일 유형",
    "priority": "우선순위",
    "reasoning": "판단 근거 설명"
}"""

# 이메일 응답 템플릿
EMAIL_TEMPLATES = {
    # 고객 문의 관련 템플릿
    "고객문의": {
        "긴급": "안녕하세요. 귀하의 긴급 문의에 대해 최우선으로 확인 중입니다. 현재 전담 상담사가 배정되어 신속히 처리하고 있으며, 최대한 빠른 시일 내에 상세한 답변을 전달드리겠습니다.",
        "높음": "안녕하세요. 문의하신 내용을 확인했으며, 신속히 답변 드리도록 하겠습니다. 현재 담당 부서에서 해당 내용을 검토 중이며, 빠른 시일 내에 상세한 답변을 준비하여 전달드리겠습니다.",
        "중간": "안녕하세요. 문의해 주셔서 감사합니다. 순차적으로 확인 후 답변 드리겠습니다. 검토에 필요한 시간이 있을 수 있으나, 최대한 상세한 답변을 드리도록 하겠습니다.",
        "낮음": "안녕하세요. 문의하신 내용에 대해 검토 후 연락 드리도록 하겠습니다. 일반적으로 3-5영업일 정도 소요될 수 있음을 양해 부탁드립니다.",
    },
    # 기술 지원 관련 템플릿
    "기술지원": {
        "긴급": "문제 상황을 확인했습니다. 기술팀이 즉시 조치를 시작했으며, 시스템 복구를 최우선으로 진행하고 있습니다. 진행 상황을 수시로 업데이트하여 전달드리겠습니다.",
        "높음": "기술적 이슈에 대해 확인했습니다. 우선순위로 검토하고 있으며, 기술팀이 문제 해결을 위해 상세 분석을 진행하고 있습니다. 오늘 중으로 진행 상황을 공유드리겠습니다.",
        "중간": "기술지원 요청을 접수했습니다. 순차적으로 처리하고 있으며, 예상 처리 시간은 1-2영업일입니다. 진행 상황은 정기적으로 업데이트하여 알려드리겠습니다.",
        "낮음": "기술지원 요청을 등록했습니다. 일정을 조율하여 지원하도록 하겠으며, 처리 가능한 일정을 확정하여 다시 연락드리겠습니다.",
    },
    # 영업 요청 관련 템플릿
    "영업요청": {
        "긴급": """안녕하세요.
긴급 요청하신 건에 대해 즉시 확인 중입니다. 
현재 담당 영업팀에 최우선 검토를 요청했으며, 1시간 이내로 구체적인 내용을 가지고 연락드리도록 하겠습니다.
신속한 지원을 위해 최선을 다하겠습니다.""",
        "높음": """안녕하세요.
요청하신 내용에 대해 담당 영업팀이 우선적으로 검토하고 있습니다.
오늘 영업일 마감 전까지 상세한 답변을 준비하여 연락드리도록 하겠습니다.
문의해 주셔서 감사합니다.""",
        "중간": """안녕하세요.
영업 관련 문의 주셔서 감사합니다.
접수된 내용을 바탕으로 최적의 제안을 준비하여 2-3영업일 내로 연락드리도록 하겠습니다.
추가 문의사항이 있으시면 언제든 말씀해 주시기 바랍니다.""",
        "낮음": """안녕하세요.
문의하신 내용이 접수되었습니다.
담당 부서와 협의하여 상세 검토 후, 다음 주 중으로 답변 드리도록 하겠습니다.
기다려 주셔서 감사합니다.""",
    },
    # 내부 협업 관련 템플릿
    "내부협업": {
        "긴급": """안녕하세요.
요청하신 협업 건에 대해 즉시 검토를 시작했습니다.
관련 부서들과 긴급 미팅을 조율 중이며, 1시간 이내로 진행 계획을 공유드리겠습니다.
신속한 협업을 위해 최선을 다하겠습니다.""",
        "높음": """안녕하세요.
협업 요청 내용 확인했습니다.
금일 중으로 관련 부서와의 논의를 완료하고, 구체적인 실행 계획을 수립하여 공유드리도록 하겠습니다.
원활한 진행을 위해 노력하겠습니다.""",
        "중간": """안녕하세요.
협업 요청 감사합니다.
이번 주 내로 관련 부서들과 미팅을 진행하고, 세부 일정 및 진행 방안을 정리하여 공유드리도록 하겠습니다.
협조해 주셔서 감사합니다.""",
        "낮음": """안녕하세요.
협업 요청 주셔서 감사합니다.
다음 주 정기 미팅 시 해당 안건을 포함하여 논의하도록 하겠습니다.
확정된 내용은 회의 후 별도로 공유드리도록 하겠습니다.""",
    },
    # 회의 일정 관련 템플릿
    "회의일정": {
        "긴급": """안녕하세요.
긴급 회의 소집 요청 확인했습니다.
현재 필수 참석자들의 일정을 최우선으로 확인 중이며, 30분 이내로 회의 시간을 확정하여 알려드리겠습니다.
회의실 예약 상황도 함께 확인하고 있습니다.
신속한 조율을 위해 노력하겠습니다.""",
        "높음": """안녕하세요.
회의 요청 주셔서 감사합니다.
오늘 중으로 참석 대상자들의 일정을 확인하여, 이번 주 내로 미팅이 진행될 수 있도록 조율하겠습니다.
확정되는 대로 회의 일정 및 장소를 공유드리도록 하겠습니다.""",
        "중간": """안녕하세요.
회의 일정 조율 요청 받았습니다.
다음 주 중으로 가능한 시간대를 조사하여 회신드리도록 하겠습니다.
참석자들의 일정을 고려하여 최적의 시간을 제안드리겠습니다.""",
        "낮음": """안녕하세요.
회의 요청 감사합니다.
월간 정기 미팅에 해당 안건을 포함하여 진행하는 것으로 검토하고 있습니다.
일정 확정 시 다시 연락드리도록 하겠습니다.""",
    },
    # 기타 일반 문의 템플릿
    "기타": {
        "긴급": """안녕하세요.
긴급하게 요청하신 건에 대해 즉시 확인하고 있습니다.
최대한 빠른 시간 내에 관련 내용을 검토하여 회신드리도록 하겠습니다.
신속한 처리를 위해 최선을 다하겠습니다.""",
        "높음": """안녕하세요.
요청하신 내용에 대해 우선적으로 확인하고 있습니다.
오늘 중으로 검토를 완료하여 상세한 답변을 전달드리도록 하겠습니다.
문의해 주셔서 감사합니다.""",
        "중간": """안녕하세요.
문의하신 내용이 접수되었습니다.
순차적으로 검토하여 2-3일 내로 답변 드리도록 하겠습니다.
기다려 주셔서 감사합니다.""",
        "낮음": """안녕하세요.
문의 주셔서 감사합니다.
접수된 내용을 검토하여 다음 주 중으로 회신드리도록 하겠습니다.
양해 부탁드립니다.""",
    },
}


def analyze_email_with_gpt(email_content: str, sender: str) -> Dict:
    """
    GPT를 사용하여 이메일 내용을 분석하는 함수

    Args:
        email_content (str): 이메일 내용
        sender (str): 발신자 정보

    Returns:
        Dict: 분석 결과 (이메일 유형, 우선순위, 판단 근거)
    """
    try:
        # GPT에게 이메일 분석 요청
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {"role": "system", "content": ANALYSIS_PROMPT},
                {"role": "user", "content": f"발신자: {sender}\n\n{email_content}"},
            ],
            response_format={"type": "json_object"},
        )

        # JSON 응답 파싱
        analysis = json.loads(response.choices[0].message.content)
        return analysis

    except Exception as e:
        print(f"이메일 분석 중 오류 발생: {e}")
        return {"type": "기타", "priority": "중간", "reasoning": "분석 중 오류 발생"}


def get_response_template(email_type: str, priority: str) -> str:
    """
    분석 결과에 따른 응답 템플릿을 반환하는 함수

    Args:
        email_type (str): 이메일 유형
        priority (str): 우선순위

    Returns:
        str: 응답 템플릿
    """
    return EMAIL_TEMPLATES.get(email_type, {}).get(
        priority, "문의하신 내용이 접수되었습니다. 확인 후 답변 드리도록 하겠습니다."
    )


def process_email(email_content: str, sender: str) -> Dict:
    """
    이메일을 처리하는 메인 함수

    Args:
        email_content (str): 이메일 내용
        sender (str): 발신자 정보

    Returns:
        Dict: 처리 결과
    """
    # 1. GPT로 이메일 분석
    analysis = analyze_email_with_gpt(email_content, sender)

    # 2. 분석 결과에 따른 응답 템플릿 가져오기
    response_template = get_response_template(analysis["type"], analysis["priority"])

    # 3. 처리 결과 반환
    return {
        "analysis": analysis,
        "sender" : sender, 
        "response_template": response_template,
        "timestamp": datetime.now().isoformat(),
    }


# 테스트 실행
if __name__ == "__main__":
    # 테스트용 이메일
    test_emails = [
        {
            "content": """
            제목: 서비스 장애 발생
            
            안녕하세요,
            현재 결제 시스템에서 간헐적인 오류가 발생하고 있습니다.
            고객들의 불만이 접수되고 있어 긴급 확인 부탁드립니다.
            
            감사합니다.
            """,
            "sender": "customer@example.com",
        },
        {
            "content": """
            제목: 제품 문의
            
            안녕하세요,
            귀사의 제품 A에 대해 자세한 정보를 알고 싶습니다.
            가격과 배송 기간에 대해 알려주시면 감사하겠습니다.
            
            감사합니다.
            """,
            "sender": "potential_customer@example.com",
        },
        
        {
            
            "content": """
            제목: 제품 사양 및 구매 절차 문의

            안녕하세요,
            귀사의 스마트홈 제품에 관심이 있어 문의 드립니다.

            1. 현재 판매 중인 IoT 허브의 상세 사양이 궁금합니다.
            2. 대량 구매 시 적용되는 할인율은 어떻게 되나요?
            3. 평균적인 배송 기간이 얼마나 걸리는지 알고 싶습니다.

            자세한 안내 부탁드립니다.
            감사합니다.

            김고객 드림
            """,
            "sender": "potential_customer2@example.com",
        }
    ]

    # 각 테스트 이메일 처리
    for idx, email in enumerate(test_emails, 1):
        print(f"\n=== 테스트 이메일 {idx} ===")
        result = process_email(email["content"], email["sender"])
        print("\n분석 결과:")
        print(f"수신된 이메일 내용: {email['content']}")
        print(f"발송자 이메일 주소: {email['sender']}")
        print(f"유형: {result['analysis']['type']}")
        print(f"우선순위: {result['analysis']['priority']}")
        print(f"판단 근거: {result['analysis']['reasoning']}")
        print("\n추천 응답:")
        print(result["response_template"])


=== 테스트 이메일 1 ===

분석 결과:
수신된 이메일 내용: 
            제목: 서비스 장애 발생
            
            안녕하세요,
            현재 결제 시스템에서 간헐적인 오류가 발생하고 있습니다.
            고객들의 불만이 접수되고 있어 긴급 확인 부탁드립니다.
            
            감사합니다.
            
발송자 이메일 주소: customer@example.com
유형: 기술지원
우선순위: 긴급
판단 근거: 이메일 내용에는 결제 시스템의 간헐적인 오류 발생이라는 기술적 문제가 언급되어 있으며, 고객들의 불만이 접수되고 있어 긴급 확인이 필요하다고 언급되어 있습니다.

추천 응답:
문제 상황을 확인했습니다. 기술팀이 즉시 조치를 시작했으며, 시스템 복구를 최우선으로 진행하고 있습니다. 진행 상황을 수시로 업데이트하여 전달드리겠습니다.

=== 테스트 이메일 2 ===

분석 결과:
수신된 이메일 내용: 
            제목: 제품 문의
            
            안녕하세요,
            귀사의 제품 A에 대해 자세한 정보를 알고 싶습니다.
            가격과 배송 기간에 대해 알려주시면 감사하겠습니다.
            
            감사합니다.
            
발송자 이메일 주소: potential_customer@example.com
유형: 고객문의
우선순위: 중간
판단 근거: 고객이 제품 A에 대한 정보, 가격, 배송 기간에 대해 문의하고 있으므로 2-3일 내에 답변이 필요합니다.

추천 응답:
안녕하세요. 문의해 주셔서 감사합니다. 순차적으로 확인 후 답변 드리겠습니다. 검토에 필요한 시간이 있을 수 있으나, 최대한 상세한 답변을 드리도록 하겠습니다.

=== 테스트 이메일 3 ===

분석 결과:
수신된 이메일 내용: 
            제목: 제품 사양 및 구매 절차 문의

     